In [1]:
from tensorflow.keras import Sequential
from  tensorflow.keras.layers import Dense, LSTM, GRU

import pandas as pd
import numpy as np 

In [2]:
window_size = 7

df = pd.read_csv("smart_power.csv")

X , y = [] , []
for i in range(len(df) - window_size):
    X.append(df["Electricity_Demand_MW"][i : i + window_size])
    y.append(df["Electricity_Demand_MW"][i + window_size])

X = np.array(X)
y = np.array(y)

In [3]:
model_1 = Sequential([
    LSTM(32, activation="tanh", 
    recurrent_activation="sigmoid", 
    input_shape=(window_size, 1)),
    Dense(1, activation="linear")])

model_1.summary()

d:\Python Programs\DevEnv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 32)             │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,385 (17.13 KB)

 Trainable params: 4,385 (17.13 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.1)  

In [5]:
model_1.compile(optimizer=optimizer, loss="mse", metrics=["mae"])


In [6]:
model_1.fit(X, y, epochs=500, batch_size=32, validation_split=.25)

Epoch 1/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 322ms/step - loss: 339016.1250 - mae: 574.7436 - val_loss: 668575.0625 - val_mae: 816.6569
Epoch 2/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 335759.9375 - mae: 571.8953 - val_loss: 664957.7500 - val_mae: 814.4393
Epoch 3/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 333228.3125 - mae: 569.6520 - val_loss: 661218.5000 - val_mae: 812.1404
Epoch 4/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 330599.4375 - mae: 567.3446 - val_loss: 657440.6875 - val_mae: 809.8112
Epoch 5/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 327964.9688 - mae: 565.0109 - val_loss: 653648.1875 - val_mae: 807.4661
Epoch 6/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 325307.4688 - mae: 562.6640 - val_loss: 649854.8125 - val_mae: 805.1138
Epoch 7/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 322679.5625 - mae: 560.3096 - val_loss: 646063.5000 - val_mae: 802.7559
Epoch 8/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - loss: 320039.3438 - mae: 557.9514 

In [12]:
model_2 = Sequential([
    GRU(32 ,
        activation="tanh",
        recurrent_activation = "sigmoid",
        input_shape=(window_size, 1)
        ),
        Dense(1)
    ])

d:\Python Programs\DevEnv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
from sklearn.preprocessing import StandardScaler


split = int(len(X) * 0.8)
X_tr, X_val = X[:split], X[split:]
y_tr, y_val = y[:split], y[split:]


y_tr = y_tr.reshape(-1, 1)   # (N, 1)
y_val = y_val.reshape(-1, 1) 


x_scaler = StandardScaler().fit(X_tr)
y_scaler = StandardScaler().fit(y_tr)


X_tr = x_scaler.transform(X_tr)
X_val = x_scaler.transform(X_val)
y_tr = y_scaler.transform(y_tr)
y_val = y_scaler.transform(y_val)

X_tr = X_tr[..., None]
X_val = X_val[..., None]


In [28]:
model_1.fit(X_tr,y_tr, epochs=500, validation_data=(X_val,y_val))

Epoch 1/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0089 - mae: 0.0703 - val_loss: 18.6844 - val_mae: 3.1579
Epoch 2/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0090 - mae: 0.0700 - val_loss: 18.7036 - val_mae: 3.1654
Epoch 3/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0088 - mae: 0.0667 - val_loss: 18.7317 - val_mae: 3.1746
Epoch 4/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0087 - mae: 0.0677 - val_loss: 18.6561 - val_mae: 3.1693
Epoch 5/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0087 - mae: 0.0672 - val_loss: 18.4931 - val_mae: 3.1512
Epoch 6/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0085 - mae: 0.0669 - val_loss: 18.0906 - val_mae: 3.1099
Epoch 7/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0089 - mae: 0.0693 - val_loss: 15.6714 - val_mae: 2.8477
Epoch 8/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0087 - mae: 0.0684 - val_loss: 15.5042 - val_mae: 2.7827
Epoch 9/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0082

In [37]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

model_2 = Sequential([
    GRU(
        32,
        activation="tanh",
        recurrent_activation="sigmoid",
        input_shape=(window_size, 1)
    ),
    Dense(1)
])

model_2.compile(optimizer="adam",loss="mse",metrics=["mae"])

d:\Python Programs\DevEnv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [38]:

import tensorflow as tf
tf.config.run_functions_eagerly(True)


In [39]:
model_2.fit(X_tr, y_tr, epochs=100,batch_size=32, validation_data=(X_val, y_val))

Epoch 1/100


d:\Python Programs\DevEnv\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - loss: 0.7265 - mae: 0.7261 - val_loss: 4.0182 - val_mae: 1.9845
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 0.6178 - mae: 0.6697 - val_loss: 3.4091 - val_mae: 1.8267
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - loss: 0.5198 - mae: 0.6129 - val_loss: 2.8319 - val_mae: 1.6635
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - loss: 0.4243 - mae: 0.5526 - val_loss: 2.3295 - val_mae: 1.5073
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - loss: 0.3371 - mae: 0.4908 - val_loss: 1.8565 - val_mae: 1.3437
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - loss: 0.2575 - mae: 0.4283 - val_loss: 1.4429 - val_mae: 1.1823
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - loss: 0.1887 - mae: 0.3651 - val_loss: 1.0956 - val_mae: 1.0274
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - loss: 0.1294 - mae: 0.2996 - val_loss: 0.8075 - val_mae: 0.8785
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 0.0808 - mae: 0.2